In [1]:
import pandas as pd
import nltk
import re
import os
from functools import reduce
from unicodedata import normalize

In [2]:
dados = pd.read_csv('data/estadao_noticias_eleicao.csv')

# Join do conteúdo
Juntando os títulos das notícias com seus respectivos conteúdos,
para posteriomente facilitar a tokenização

In [3]:
def limpar_texto(texto):
    pattern = re.compile('[^a-zA-Z0-9 ]')
    texto = normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    return pattern.sub(' ', texto)

In [4]:
materias = dados.titulo + " " + dados.subTitulo +  " " + dados.conteudo
materias = materias.apply(lambda texto: "" if isinstance(texto, float) else limpar_texto(texto))
ids = dados.idNoticia
    

# Tokenizando conteúdo
Criando tokens com cada palavra do texto para que posteriormente possam ser indexadas e associadas aos respectivos ids das notícias

In [5]:
tokens = materias.apply(nltk.word_tokenize)

# Indexando tokens
Criando indices invertidos com os tokens para poder aplicar os métodos de busca 

In [6]:
class Termo(object):
    def __init__(self, termo):
        self.termo = termo
        self.idf = 1
        
    def __hash__(self):
        return hash(self.termo)
        
    def __eq__(self, other):
        if isinstance(other, str):
            return self.termo == other
        return self.termo == other.termo
    
    def __str__(self):
        return "Termo: %s" %(self.termo) + os.linesep + "Idf: %d" %(self.idf)

In [7]:
# index = {}

# for i in range(len(tokens)):
#     id_noticia = ids[i]
#     palavras = tokens[i]
#     for palavra in palavras:
#         palavra = Termo(palavra.lower())
#         if palavra not in index:
#             index[palavra] = []
        
#         id_rec = reduce(lambda found, id_not: id_not if id_not[1] == id_noticia else found, index.get(palavra, []), None)
        
#         if not id_rec:
#             index[palavra].append((1, id_noticia))
#         else:
#             index_id = index[palavra].index(id_rec)
#             index[palavra][index_id] = (id_rec[0] + 1, id_rec[1])

In [8]:
index = {}

for i in range(len(tokens)):
    id_noticia = ids[i]
    palavras = tokens[i]
    for palavra in palavras:
        palavra = palavra.lower()
        if palavra not in index:
            index[palavra] = (0, [])
        
        id_rec = reduce(lambda found, id_not: id_not if id_not[1] == id_noticia else found, index.get(palavra, (-1, []))[1], None)
        
        if not id_rec:
            docs = index[palavra]
            docs[1].append((1, id_noticia))
            index[palavra] = (docs[0] + 1, docs[1])
        else:
            docs = index[palavra]
            index_id = docs[1].index(id_rec)
            docs[1][index_id] = (id_rec[0] + 1, id_rec[1])
            index[palavra] = (docs[0] + 1, docs[1])

KeyboardInterrupt: 

In [ ]:
print(len(index))